In [1]:
import pandas as pd
import numpy as np
import os
import sys
from PIL import Image
import random
import tensorflow
import matplotlib.pyplot as plt
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization, Reshape, LSTM
from keras.losses import categorical_crossentropy
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
import tensorflow
import keras


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
AUTOTUNE = tensorflow.data.experimental.AUTOTUNE

In [4]:
# We will traverse through Project_data and build 2 DataFrames each for training and validation with below 4 columns
#  Image_File_name | Image_Folder_Name | Image/Gesture_Class_Name | Image/Gesture_Class_label/numerical

# Let us read train.csv and val.csv to build -  folder_name | Gesture_Name | Gesture_Class_label first.

ROOT_PATH = "/content/drive/My Drive/Gesture_Assignment/Project_data"
TRAIN_MAPPING = ROOT_PATH + "/train.csv"
VAL_MAPPING = ROOT_PATH + "/val.csv"
#img_width, img_height = 180, 180
batch_size = 32
# we have 30 images, but we chose 24 below and we pick them randomly and sort to ensure temporal dependence
num_samples = 28
num_of_channels = 3
num_of_classes = 5
    
    
train_map_df = pd.read_csv(TRAIN_MAPPING,sep=";",header=None)
val_map_df = pd.read_csv(VAL_MAPPING,sep=";",header=None)
DF_COLS = ["Folder_Name", "Gesture_Name", "Gesture_Class"]

train_map_df.columns = DF_COLS
val_map_df.columns = DF_COLS

print(train_map_df.tail())
print(val_map_df.tail())


                                 Folder_Name   Gesture_Name  Gesture_Class
658  WIN_20180907_16_38_23_Pro_Thumbs Up_new  Thumbs Up_new              4
659  WIN_20180907_16_41_09_Pro_Thumbs Up_new  Thumbs Up_new              4
660  WIN_20180907_16_42_05_Pro_Thumbs Up_new  Thumbs Up_new              4
661  WIN_20180907_16_42_55_Pro_Thumbs Up_new  Thumbs Up_new              4
662  WIN_20180907_16_43_39_Pro_Thumbs Up_new  Thumbs Up_new              4
                                Folder_Name   Gesture_Name  Gesture_Class
95  WIN_20180907_15_43_36_Pro_Thumbs Up_new  Thumbs Up_new              4
96  WIN_20180907_15_52_05_Pro_Thumbs Up_new  Thumbs Up_new              4
97  WIN_20180907_15_54_30_Pro_Thumbs Up_new  Thumbs Up_new              4
98  WIN_20180907_16_10_59_Pro_Thumbs Up_new  Thumbs Up_new              4
99  WIN_20180907_16_39_59_Pro_Thumbs Up_new  Thumbs Up_new              4


In [5]:
def create_mapping_dataframe(dir_name,input_df):
    
    '''
    We call this function with 'train' and 'val' as dir_name i.e. directory names argument.
    and input_df which has three columns so far. We navigate through each image folder and build data frame.
    This data frame will have 4 columns indicated by df_columns. This dataframe which we return then becomes
    master referrence throughout the program.
    
    Arguments : 
      directory_name - train and val 
      dataframe - created in earlier step with mapping of folder, gesture name and class 
    Returns :
      dataframe - with image file names populated
    '''
    
    train_image_mapping = []
    df_columns = ["Image_File", "Folder_Name", "Gesture_Name", "Gesture_Class"]

    for each_folder in input_df['Folder_Name'].values:
    
        e_gesture_name = input_df.loc[input_df['Folder_Name'] == each_folder ,'Gesture_Name'].values[0]
        e_gesture_class = input_df.loc[input_df['Folder_Name'] == each_folder ,'Gesture_Class'].values[0]
    
        folder_full_path = ROOT_PATH + "/" + dir_name + "/"  + each_folder
        for each_file in os.listdir(folder_full_path):
            train_image_mapping.append((each_file, each_folder, e_gesture_name, e_gesture_class))
            
    return_df = pd.DataFrame(data=train_image_mapping, columns=df_columns)
    
    return return_df

In [6]:
def convert_images_to_numpy(image_files, folder, ops_mode):
    
    '''
    Given the image file names, folder strucutre resize, and convert into numpy array.
    Arguments:
      image_files - list of image files coming from some folder
      folder - folder name after ROOT_PATH
      ops_mode - train or val to indicate location of images
    Returns:
      imagelist in multidimensional numpy array 
    '''
    
    return_images = np.zeros((len(image_files),img_width,img_height,3))
    folder_full_path = ROOT_PATH + "/" + ops_mode + "/"  + folder
    
    count = 0
    for each_file in image_files: 
        img = Image.open(os.path.join(folder_full_path, each_file))
        img = img.resize((img_width, img_height))
        img = np.asarray(img)
        img = img/255
        img = np.round(img,3)
        return_images[count] = img
        count += 1
    
    return(return_images)

In [7]:
def convert_all_images_to_numpy(df, ops_mode):
    
    '''
    Given the image folder, folder strucutre resize, and convert into numpy array.
    Arguments:
      df - Dataframe with mapping of folders, image files 
      ops_mode - train or val to indicate location of images
    Returns:
      imagelist in dictionary in multidimensional numpy array 
    '''

    grouped_df = df.groupby('Folder_Name')
    folder_list = grouped_df['Folder_Name'].unique().values.flatten()
    
    image_dict = {}

    count = 0
    for folders in folder_list:
        each_folder = folders[0]
        folder_full_path = ROOT_PATH + "/" + ops_mode + "/"  + each_folder
        file_list_df = grouped_df.get_group(each_folder)['Image_File']
        file_list_tuple = list(zip(file_list_df.index.values, file_list_df.values))
        for each_tuple in file_list_tuple:
            img = Image.open(os.path.join(folder_full_path, each_tuple[1]))
            img = img.resize((img_width, img_height))
            img = np.asarray(img)
            img = img/255
            img = np.round(img,3)
            image_dict[each_tuple[0]] = img
        print("Finished processing of folder {}".format(count))
        count += 1

    return image_dict

In [8]:
def convert_images_to_numpy_from_dict(image_files, image_dict, df):
    
    '''
    Given the image file names, folder strucutre resize, and convert into numpy array.
    Arguments:
      image_files - list of image files coming from some folder
      folder - folder name after ROOT_PATH
      ops_mode - train or val to indicate location of images
    Returns:
      imagelist in multidimensional numpy array 
    '''
    grouped_df = df.groupby('Image_File')

    return_images = np.zeros((len(image_files),img_width,img_height,3))
    count = 0 
    for each_file in image_files:
        file_idx = grouped_df.get_group(each_file).index.values[0]
        file_idx = str(file_idx)
        #print(file_idx , type(file_idx))
        #x = image_dict[file_idx]
        return_images[count] = image_dict[file_idx]
    
    return(return_images)

In [9]:
def batch_generator(df, batch_size, ops_mode, image_dict):
    
    '''
    Generate the batch yielding X and y for training via CNN and LSTM.
    Arguments:
      df - dataframe with image_names , and other details like folder names , mapping to gestures etc.
      batch_size - int value indicating how much we want to fit into one batch
      ops_mode - train or val to indicate if we are calling generator for training or validation.
        this operation will change folder structure to refer to 
    Returns:
      Yields batch with X and y values
    '''
    

    batch_array_size_tuple = (batch_size, num_samples, img_width, img_height, num_of_channels)

    while True:
        
        # Let us shuffle DF to get different sequence of images in every batch
        shuffled_df = df.groupby('Folder_Name', group_keys=False).apply(lambda x: x.sample(frac=1))
        folder_list = shuffled_df['Folder_Name'].unique()
        
        # first for loop below creates batchs and also ensures any residual rows are placed in the last element of array
        # batch X values are of the shape [ batch_size, num_of_samples_per_folder, image_width, image_height, channels ]
        # Batch y values are of the shape [ batch_size, class_labels_in_one_hot ]
        
        folder_batches = []
        for i in range(0, len(folder_list), batch_size):
            folder_batches.append(folder_list[i:i+batch_size])
        
        # Let us iterate through batches and yield X and y for each batch
        for each_batch in folder_batches:
            batch_data = np.zeros(batch_array_size_tuple)
            batch_labels = np.zeros((batch_size,5))

            # we ignore first and last 2 frames in sequences thinking it may not have captured anything meaningful                        
            #random_imglist = random.sample(range(0, 30), num_samples)
            #random_imglist = sorted(random_imglist)
            # we ignore first seueqnce thinking it may not have captured anything meaningful 
            random_imglist = range(1,num_samples+1)
            
            folder_count = 0
            for each_folder in each_batch:
                img_files = sorted(shuffled_df.groupby('Folder_Name')
                                   .get_group(each_folder)[["Image_File"]]
                                   .values.flatten()
                                  )
                img_files = [img_files[i] for i in random_imglist]
                #temp_data = convert_images_to_numpy(img_files, each_folder, ops_mode)
                temp_data = convert_images_to_numpy_from_dict(img_files, image_dict, df)
                batch_data[folder_count] = temp_data
                gesture = int(shuffled_df.groupby('Folder_Name')
                              .get_group(each_folder)['Gesture_Class'].values[0]
                             )
                batch_labels[folder_count, gesture] = 1
                folder_count += 1

            #print("Completed batch {}".format(batch_data.shape))
            yield batch_data, batch_labels


In [10]:
# we create master mapping and store it in dataframe
new_train_map_df = create_mapping_dataframe('train', train_map_df)
new_val_map_df = create_mapping_dataframe('val', val_map_df)

train_image_dict_file = ROOT_PATH + "/" + "train_dict_2.npz"
val_image_dict_file = ROOT_PATH + "/" + "val_dict_2.npz"

train_image_dict = np.load(train_image_dict_file)
val_image_dict = np.load(val_image_dict_file)

In [11]:
print(train_image_dict['15570'],train_image_dict['15570'].shape)

[[[0.471 0.463 0.439]
  [0.467 0.463 0.424]
  [0.467 0.467 0.408]
  ...
  [0.541 0.522 0.475]
  [0.525 0.529 0.475]
  [0.518 0.518 0.471]]

 [[0.471 0.475 0.447]
  [0.471 0.475 0.439]
  [0.471 0.475 0.42 ]
  ...
  [0.533 0.518 0.475]
  [0.522 0.518 0.467]
  [0.51  0.514 0.463]]

 [[0.51  0.514 0.482]
  [0.51  0.514 0.478]
  [0.51  0.514 0.467]
  ...
  [0.529 0.51  0.467]
  [0.518 0.514 0.463]
  [0.506 0.514 0.463]]

 ...

 [[0.506 0.439 0.4  ]
  [0.341 0.259 0.235]
  [0.345 0.243 0.212]
  ...
  [0.282 0.204 0.196]
  [0.271 0.2   0.184]
  [0.235 0.188 0.173]]

 [[0.341 0.286 0.251]
  [0.129 0.075 0.067]
  [0.176 0.106 0.102]
  ...
  [0.267 0.2   0.192]
  [0.235 0.192 0.18 ]
  [0.184 0.157 0.149]]

 [[0.161 0.141 0.118]
  [0.075 0.059 0.063]
  [0.082 0.039 0.067]
  ...
  [0.255 0.188 0.188]
  [0.216 0.169 0.165]
  [0.141 0.122 0.125]]] (120, 160, 3)


In [ ]:
'''
train_image_dict = {}
val_image_dict = {}
train_image_dict = convert_all_images_to_numpy(new_train_map_df, 'train')
val_image_dict = convert_all_images_to_numpy(new_val_map_df, 'val')

train_image_dict_file = ROOT_PATH + "/" + "train_dict.npz"
val_image_dict_file = ROOT_PATH + "/" + "val_dict.npz"

train_image_dict_str = {str(key): value for key, value in train_image_dict.items()}
np.savez_compressed(train_image_dict_file, **train_image_dict_str)
val_image_dict_str = {str(key): value for key, value in val_image_dict.items()}
np.savez_compressed(val_image_dict_file, **val_image_dict_str)

'''

'\ntrain_image_dict = {}\nval_image_dict = {}\ntrain_image_dict = convert_all_images_to_numpy(new_train_map_df, \'train\')\nval_image_dict = convert_all_images_to_numpy(new_val_map_df, \'val\')\n\ntrain_image_dict_file = ROOT_PATH + "/" + "train_dict.npz"\nval_image_dict_file = ROOT_PATH + "/" + "val_dict.npz"\n\ntrain_image_dict_str = {str(key): value for key, value in train_image_dict.items()}\nnp.savez_compressed(train_image_dict_file, **train_image_dict_str)\nval_image_dict_str = {str(key): value for key, value in val_image_dict.items()}\nnp.savez_compressed(val_image_dict_file, **val_image_dict_str)\n\n'

In [12]:
train_size = len(train_map_df)
val_size = len(val_map_df)

if (train_size%batch_size) == 0:
    steps_per_epoch = int(train_size/batch_size)
else:
    steps_per_epoch = (train_size//batch_size) + 1

if (val_size%batch_size) == 0:
    validation_steps = int(val_size/batch_size)
else:
    validation_steps = (val_size//batch_size) + 1

In [13]:
print(train_size,val_size,steps_per_epoch,validation_steps)

663 100 21 4


In [ ]:
#print(new_train_map_df.head(), new_val_map_df.tail)

In [14]:
train_batch_generator =  batch_generator(new_train_map_df, batch_size, 'train', train_image_dict)
val_batch_generator =  batch_generator(new_val_map_df, batch_size, 'val', val_image_dict)

In [ ]:
#shuffled_df = new_train_map_df.groupby('Folder_Name', group_keys=False).apply(lambda x: x.sample(frac=1))
#print(shuffled_df, type(shuffled_df))

In [ ]:
# Let us try video classification using conv3D model

input_size_tuple = (num_samples, img_width, img_height, num_of_channels)

# input layer
input_layer = Input(input_size_tuple)
# Conv layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu')(conv_layer1)
# max pooling layer
pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer2)

conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(pooling_layer1)
conv_layer4 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(conv_layer3)
pooling_layer2 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer4)

#pooling_layer2 = BatchNormalization()(pooling_layer2)
flatten_layer = Flatten()(pooling_layer2)

## add dropouts to avoid overfitting / perform regularization
dense_layer1 = Dense(units=128, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.2)(dense_layer1)
dense_layer2 = Dense(units=64, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.2)(dense_layer2)
output_layer = Dense(units=num_of_classes, activation='softmax')(dense_layer1)

model = Model(inputs=input_layer, outputs=output_layer)



In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 180, 180, 3)  0         
                             ]                                   
                                                                 
 conv3d (Conv3D)             (None, 26, 178, 178, 8)   656       
                                                                 
 conv3d_1 (Conv3D)           (None, 24, 176, 176, 16)  3472      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 12, 88, 88, 16)   0         
 )                                                               
                                                                 
 conv3d_2 (Conv3D)           (None, 10, 86, 86, 32)    13856     
                                                                 
 conv3d_3 (Conv3D)           (None, 8, 84, 84, 64)     55360 

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
history = History()

epochs = 20

# let us create checkpoint so that weights are saved after every epoch
checkpoint_filepath = ROOT_PATH + "/" + "ccn_checkpoint_1.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, save_weights_only=True)


In [ ]:
'''
#with tensorflow.device('/CPU:0'):
model.fit(train_batch_generator, validation_data=val_batch_generator, 
                steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, 
                verbose=1, callbacks=[checkpoint, history, reduce_lr])
'''

"\n#with tensorflow.device('/CPU:0'):\nmodel.fit(train_batch_generator, validation_data=val_batch_generator, \n                steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, \n                verbose=1, callbacks=[checkpoint, history, reduce_lr])\n"

In [ ]:
'''
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
'''

"\nacc = history.history['acc']\nval_acc = history.history['val_acc']\n\nloss = history.history['loss']\nval_loss = history.history['val_loss']\n\nepochs_range = range(epochs)\n\nplt.figure(figsize=(8, 8))\nplt.subplot(1, 2, 1)\nplt.plot(epochs_range, acc, label='Training Accuracy')\nplt.plot(epochs_range, val_acc, label='Validation Accuracy')\nplt.legend(loc='lower right')\nplt.title('Training and Validation Accuracy')\n\nplt.subplot(1, 2, 2)\nplt.plot(epochs_range, loss, label='Training Loss')\nplt.plot(epochs_range, val_loss, label='Validation Loss')\nplt.legend(loc='upper right')\nplt.title('Training and Validation Loss')\nplt.show()\n"

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0005)
opt = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
history = History()

epochs = 40

# let us create checkpoint so that weights are saved after every epoch
checkpoint_filepath = ROOT_PATH + "/" + "ccn_checkpoint_2.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, save_weights_only=True)

In [ ]:
#with tensorflow.device('/CPU:0'):
model.fit(train_batch_generator, validation_data=val_batch_generator, 
                steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, 
                verbose=1, callbacks=[checkpoint, history, reduce_lr])

Epoch 1/40
21/21 [==============================] - 128s 6s/step - loss: 1.5862 - acc: 0.2068 - val_loss: 1.2351 - val_acc: 0.2031 - lr: 5.0000e-04
Epoch 2/40
21/21 [==============================] - 120s 6s/step - loss: 1.5028 - acc: 0.2976 - val_loss: 1.1369 - val_acc: 0.3203 - lr: 5.0000e-04
Epoch 3/40
21/21 [==============================] - 120s 6s/step - loss: 1.3251 - acc: 0.4464 - val_loss: 0.9238 - val_acc: 0.3672 - lr: 5.0000e-04
Epoch 4/40
21/21 [==============================] - 117s 6s/step - loss: 1.0998 - acc: 0.5417 - val_loss: 0.8685 - val_acc: 0.3984 - lr: 5.0000e-04
Epoch 5/40
21/21 [==============================] - 119s 6s/step - loss: 0.9223 - acc: 0.6458 - val_loss: 0.9872 - val_acc: 0.3906 - lr: 5.0000e-04
Epoch 6/40
21/21 [==============================] - 119s 6s/step - loss: 0.7471 - acc: 0.7351 - val_loss: 1.0006 - val_acc: 0.4219 - lr: 5.0000e-04
Epoch 7/40
21/21 [==============================] - 120s 6s/step - loss: 0.5867 - acc: 0.7738 - val_loss: 0.9430

KeyboardInterrupt: ignored

In [ ]:

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Let us try video classification using conv3D model - simplified model

input_size_tuple = (num_samples, img_width, img_height, num_of_channels)

# input layer
input_layer = Input(input_size_tuple)
# Conv layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu')(conv_layer1)
# max pooling layer
pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer2)

flatten_layer = Flatten()(pooling_layer1)

## add dropouts to avoid overfitting / perform regularization
dense_layer2 = Dense(units=64, activation='relu')(flatten_layer)
dense_layer2 = Dropout(0.2)(dense_layer2)
output_layer = Dense(units=num_of_classes, activation='softmax')(dense_layer2)

model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0005)
opt = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
history = History()

epochs = 30

# let us create checkpoint so that weights are saved after every epoch
checkpoint_filepath = ROOT_PATH + "/" + "ccn_checkpoint_3.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, save_weights_only=True)

In [ ]:
#with tensorflow.device('/CPU:0'):
model.fit(train_batch_generator, validation_data=val_batch_generator, 
                steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, 
                verbose=1, callbacks=[checkpoint, history, reduce_lr])

Epoch 1/30
21/21 [==============================] - 120s 6s/step - loss: 1.5988 - acc: 0.2202 - val_loss: 1.1992 - val_acc: 0.2188 - lr: 5.0000e-04
Epoch 2/30
21/21 [==============================] - 120s 6s/step - loss: 1.4499 - acc: 0.3244 - val_loss: 1.1266 - val_acc: 0.2812 - lr: 5.0000e-04
Epoch 3/30
21/21 [==============================] - 120s 6s/step - loss: 1.2946 - acc: 0.4732 - val_loss: 1.0103 - val_acc: 0.3594 - lr: 5.0000e-04
Epoch 4/30
21/21 [==============================] - 117s 6s/step - loss: 1.0801 - acc: 0.5685 - val_loss: 0.9032 - val_acc: 0.4453 - lr: 5.0000e-04
Epoch 5/30
21/21 [==============================] - 120s 6s/step - loss: 0.8117 - acc: 0.7098 - val_loss: 0.8730 - val_acc: 0.4531 - lr: 5.0000e-04
Epoch 6/30
21/21 [==============================] - 121s 6s/step - loss: 0.6892 - acc: 0.7753 - val_loss: 1.2269 - val_acc: 0.3438 - lr: 5.0000e-04
Epoch 7/30
21/21 [==============================] - 121s 6s/step - loss: 0.5988 - acc: 0.7902 - val_loss: 0.9018

In [ ]:
# Let us try video classification using conv3D model 
# Image size is 120 X 160

img_width, img_height = 120, 160
input_size_tuple = (num_samples, img_width, img_height, num_of_channels)

# input layer
input_layer = Input(input_size_tuple)
# Conv layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu')(conv_layer1)
# max pooling layer
pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer2)

conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(pooling_layer1)
conv_layer4 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(conv_layer3)
pooling_layer2 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer4)

#pooling_layer2 = BatchNormalization()(pooling_layer2)
flatten_layer = Flatten()(pooling_layer2)

## add dropouts to avoid overfitting / perform regularization
dense_layer1 = Dense(units=128, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.2)(dense_layer1)
dense_layer2 = Dense(units=64, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.2)(dense_layer2)
output_layer = Dense(units=num_of_classes, activation='softmax')(dense_layer2)

model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0005)
opt = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
history = History()

epochs = 30

# let us create checkpoint so that weights are saved after every epoch
checkpoint_filepath = ROOT_PATH + "/" + "ccn_checkpoint_4.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, save_weights_only=True)

In [ ]:
#with tensorflow.device('/CPU:0'):
model.fit(train_batch_generator, validation_data=val_batch_generator, 
                steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, 
                verbose=1, callbacks=[checkpoint, history, reduce_lr])

Epoch 1/30
21/21 [==============================] - 103s 5s/step - loss: 1.5867 - acc: 0.2232 - val_loss: 1.2433 - val_acc: 0.2031 - lr: 5.0000e-04
Epoch 2/30
21/21 [==============================] - 93s 5s/step - loss: 1.5245 - acc: 0.2768 - val_loss: 1.1488 - val_acc: 0.3281 - lr: 5.0000e-04
Epoch 3/30
21/21 [==============================] - 93s 5s/step - loss: 1.3517 - acc: 0.4390 - val_loss: 0.9854 - val_acc: 0.3750 - lr: 5.0000e-04
Epoch 4/30
21/21 [==============================] - 90s 5s/step - loss: 1.1740 - acc: 0.4940 - val_loss: 0.8864 - val_acc: 0.4219 - lr: 5.0000e-04
Epoch 5/30
21/21 [==============================] - 93s 5s/step - loss: 1.0169 - acc: 0.6012 - val_loss: 0.8697 - val_acc: 0.4375 - lr: 5.0000e-04
Epoch 6/30
21/21 [==============================] - 93s 5s/step - loss: 0.8942 - acc: 0.6562 - val_loss: 0.8304 - val_acc: 0.4453 - lr: 5.0000e-04
Epoch 7/30
21/21 [==============================] - 93s 5s/step - loss: 0.7712 - acc: 0.6949 - val_loss: 0.8050 - val

In [ ]:

batch_size = 32
# we have 30 images, but we chose 24 below and we pick them randomly and sort to ensure temporal dependence
num_samples = 28
num_of_channels = 3
num_of_classes = 5

img_width, img_height = 120, 160

# Let us try video classification using conv3D model - another method

input_size_tuple = (num_samples, img_width, img_height, num_of_channels)

# input layer
input_layer = Input(input_size_tuple)
# Conv layers
conv_layer1 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(input_layer)
#batchnorm_layer1 = BatchNormalization()(conv_layer1)
pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer1)

conv_layer2 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(pooling_layer1)
#batchnorm_layer2 = BatchNormalization()(conv_layer2)
pooling_layer2 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer2)

conv_layer3 = Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu')(pooling_layer2)
#batchnorm_layer3 = BatchNormalization()(conv_layer3)
pooling_layer3 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer3)

#pooling_layer2 = BatchNormalization()(pooling_layer2)
flatten_layer = Flatten()(pooling_layer3)

## add dropouts to avoid overfitting / perform regularization
dense_layer1 = Dense(units=512, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.3)(dense_layer1)
dense_layer2 = Dense(units=256, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.3)(dense_layer2)
output_layer = Dense(units=num_of_classes, activation='softmax')(dense_layer2)

model = Model(inputs=input_layer, outputs=output_layer)


In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0005)
opt = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
history = History()

epochs = 15

# let us create checkpoint so that weights are saved after every epoch
checkpoint_filepath = ROOT_PATH + "/" + "ccn_checkpoint_5.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, save_weights_only=True)

In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 120, 160, 3)  0         
                             ]                                   
                                                                 
 conv3d_9 (Conv3D)           (None, 26, 118, 158, 32)  2624      
                                                                 
 max_pooling3d_9 (MaxPooling  (None, 13, 59, 79, 32)   0         
 3D)                                                             
                                                                 
 conv3d_10 (Conv3D)          (None, 11, 57, 77, 64)    55360     
                                                                 
 max_pooling3d_10 (MaxPoolin  (None, 5, 28, 38, 64)    0         
 g3D)                                                            
                                                           

In [ ]:
#with tensorflow.device('/CPU:0'):
model.fit(train_batch_generator, validation_data=val_batch_generator, 
                steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, 
                verbose=1, callbacks=[checkpoint, history, reduce_lr])

Epoch 1/15
21/21 [==============================] - 47s 2s/step - loss: 1.5794 - acc: 0.1815 - val_loss: 1.3069 - val_acc: 0.1719 - lr: 5.0000e-04
Epoch 2/15
21/21 [==============================] - 46s 2s/step - loss: 1.6071 - acc: 0.1994 - val_loss: 1.2778 - val_acc: 0.1875 - lr: 5.0000e-04
Epoch 3/15
21/21 [==============================] - 47s 2s/step - loss: 1.5523 - acc: 0.2113 - val_loss: 1.5725 - val_acc: 0.2656 - lr: 5.0000e-04
Epoch 4/15
21/21 [==============================] - 46s 2s/step - loss: 1.5400 - acc: 0.2589 - val_loss: 1.2099 - val_acc: 0.2969 - lr: 5.0000e-04
Epoch 5/15
21/21 [==============================] - 46s 2s/step - loss: 1.4086 - acc: 0.3661 - val_loss: 1.0733 - val_acc: 0.3906 - lr: 5.0000e-04
Epoch 6/15
21/21 [==============================] - 46s 2s/step - loss: 1.4011 - acc: 0.3363 - val_loss: 1.3670 - val_acc: 0.3438 - lr: 5.0000e-04
Epoch 7/15
21/21 [==============================] - 47s 2s/step - loss: 1.2716 - acc: 0.3780 - val_loss: 1.2828 - val_

In [ ]:

# we have 30 images, but we chose 24 below and we pick them randomly and sort to ensure temporal dependence
num_samples = 28
num_of_channels = 3
num_of_classes = 5
batch_size = 32

img_width, img_height = 120, 160

# Let us try video classification using conv3D model - another method

input_size_tuple = (num_samples, img_width, img_height, num_of_channels)

# input layer
input_layer = Input(input_size_tuple)
# Conv layers
conv_layer1 = Conv3D(filters=8, kernel_size=(2, 2, 2), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(2, 2, 2), activation='relu')(conv_layer1)
# max pooling layer
pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer2)

conv_layer3 = Conv3D(filters=32, kernel_size=(2, 2, 2), activation='relu')(pooling_layer1)
conv_layer4 = Conv3D(filters=64, kernel_size=(2, 2, 2), activation='relu')(conv_layer3)
pooling_layer2 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer4)

#pooling_layer2 = BatchNormalization()(pooling_layer2)

flatten_layer = Flatten()(pooling_layer2)
print(flatten_layer.shape)

timesteps = 28
input_dim = flatten_layer.shape[1] // timesteps

# Reshape to (batch_size, timesteps, input_dim)


flatten_layer = Reshape((timesteps, input_dim) )(flatten_layer)

lstm_layer1 = LSTM(64, return_sequences=True)(flatten_layer)
lstm_layer2 = LSTM(16, return_sequences=False)(lstm_layer1)


## add dropouts to avoid overfitting / perform regularization
dense_layer1 = Dense(units=128, activation='relu')(lstm_layer2)
dense_layer1 = Dropout(0.4)(dense_layer1)
#dense_layer2 = Dense(units=64, activation='relu')(dense_layer1)
#dense_layer2 = Dropout(0.4)(dense_layer2)

output_layer = Dense(units=num_of_classes, activation='softmax')(dense_layer1)
model = Model(inputs=input_layer, outputs=output_layer)


(None, 340480)


In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0005)
opt = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
history = History()

epochs = 15

# let us create checkpoint so that weights are saved after every epoch
checkpoint_filepath = ROOT_PATH + "/" + "ccn_checkpoint_5.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, save_weights_only=True)

In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 28, 120, 160, 3)  0         
                             ]                                   
                                                                 
 conv3d_9 (Conv3D)           (None, 26, 118, 158, 32)  2624      
                                                                 
 max_pooling3d_9 (MaxPooling  (None, 13, 59, 79, 32)   0         
 3D)                                                             
                                                                 
 conv3d_10 (Conv3D)          (None, 11, 57, 77, 64)    55360     
                                                                 
 max_pooling3d_10 (MaxPoolin  (None, 5, 28, 38, 64)    0         
 g3D)                                                            
                                                           

In [ ]:
#with tensorflow.device('/CPU:0'):
model.fit(train_batch_generator, validation_data=val_batch_generator, 
                steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, 
                verbose=1, callbacks=[checkpoint, history, reduce_lr])

Epoch 1/15
21/21 [==============================] - 93s 5s/step - loss: 0.9778 - acc: 0.5938 - val_loss: 0.9690 - val_acc: 0.4297 - lr: 5.0000e-04
Epoch 2/15
21/21 [==============================] - 90s 4s/step - loss: 0.8804 - acc: 0.6071 - val_loss: 1.0993 - val_acc: 0.3594 - lr: 5.0000e-04
Epoch 3/15
21/21 [==============================] - 90s 4s/step - loss: 0.8446 - acc: 0.6473 - val_loss: 1.0870 - val_acc: 0.4297 - lr: 5.0000e-04
Epoch 4/15
21/21 [==============================] - 88s 4s/step - loss: 0.7554 - acc: 0.6815 - val_loss: 1.2429 - val_acc: 0.4062 - lr: 5.0000e-04
Epoch 5/15
21/21 [==============================] - 90s 5s/step - loss: 0.7113 - acc: 0.7143 - val_loss: 1.2885 - val_acc: 0.3828 - lr: 5.0000e-04
Epoch 6/15
21/21 [==============================] - 90s 5s/step - loss: 0.6507 - acc: 0.7277 - val_loss: 1.4658 - val_acc: 0.3828 - lr: 5.0000e-04
Epoch 7/15
21/21 [==============================] - 90s 5s/step - loss: 0.5875 - acc: 0.7470 - val_loss: 1.2710 - val_

In [ ]:

# we have 30 images, but we chose 24 below and we pick them randomly and sort to ensure temporal dependence
num_samples = 28
num_of_channels = 3
num_of_classes = 5
batch_size =16

img_width, img_height = 120, 160

# Let us try video classification using conv3D model - another method

input_size_tuple = (num_samples, img_width, img_height, num_of_channels)

# input layer
input_layer = Input(input_size_tuple)
# Conv layers
conv_layer1 = Conv3D(filters=8, kernel_size=(2, 2, 2), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(2, 2, 2), activation='relu')(conv_layer1)
# max pooling layer
pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer2)

conv_layer3 = Conv3D(filters=32, kernel_size=(2, 2, 2), activation='relu')(pooling_layer1)
conv_layer4 = Conv3D(filters=64, kernel_size=(2, 2, 2), activation='relu')(conv_layer3)
pooling_layer2 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer4)

#pooling_layer2 = BatchNormalization()(pooling_layer2)

flatten_layer = Flatten()(pooling_layer2)
print(flatten_layer.shape)

timesteps = 5
input_dim = flatten_layer.shape[1] // timesteps

# Reshape to (batch_size, timesteps, input_dim)


flatten_layer = Reshape((timesteps, input_dim) )(flatten_layer)

lstm_layer1 = LSTM(128, return_sequences=True)(flatten_layer)
lstm_layer2 = LSTM(32, return_sequences=False)(lstm_layer1)


## add dropouts to avoid overfitting / perform regularization
dense_layer1 = Dense(units=128, activation='relu')(lstm_layer2)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=64, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)

output_layer = Dense(units=num_of_classes, activation='softmax')(dense_layer2)
model = Model(inputs=input_layer, outputs=output_layer)


(None, 340480)


In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0005)
opt = keras.optimizers.Adam(learning_rate=0.0005)

model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['acc'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
history = History()

epochs = 15

# let us create checkpoint so that weights are saved after every epoch
checkpoint_filepath = ROOT_PATH + "/" + "ccn_checkpoint_6.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, save_weights_only=True)

In [ ]:
#with tensorflow.device('/CPU:0'):
model.fit(train_batch_generator, validation_data=val_batch_generator, 
                steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, 
                verbose=1, callbacks=[checkpoint, history, reduce_lr])

Epoch 1/15
21/21 [==============================] - 113s 5s/step - loss: 1.5861 - acc: 0.1949 - val_loss: 1.2519 - val_acc: 0.3594 - lr: 5.0000e-04
Epoch 2/15
21/21 [==============================] - 91s 5s/step - loss: 1.5610 - acc: 0.2202 - val_loss: 1.2135 - val_acc: 0.1875 - lr: 5.0000e-04
Epoch 3/15
21/21 [==============================] - 91s 5s/step - loss: 1.4916 - acc: 0.2946 - val_loss: 1.3017 - val_acc: 0.1641 - lr: 5.0000e-04
Epoch 4/15
21/21 [==============================] - 90s 4s/step - loss: 1.6233 - acc: 0.1905 - val_loss: 1.2571 - val_acc: 0.1719 - lr: 5.0000e-04
Epoch 5/15
21/21 [==============================] - 91s 5s/step - loss: 1.5921 - acc: 0.2009 - val_loss: 1.2501 - val_acc: 0.1953 - lr: 5.0000e-04
Epoch 6/15
21/21 [==============================] - 91s 5s/step - loss: 1.5603 - acc: 0.3095 - val_loss: 1.1862 - val_acc: 0.2969 - lr: 5.0000e-04
Epoch 7/15
21/21 [==============================] - 91s 5s/step - loss: 1.4643 - acc: 0.3393 - val_loss: 1.0800 - val

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
history = History()

epochs = 25

# let us create checkpoint so that weights are saved after every epoch
checkpoint_filepath = ROOT_PATH + "/" + "ccn_checkpoint_7.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, save_weights_only=True)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.01)
opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['acc'])

In [ ]:
#with tensorflow.device('/CPU:0'):
model.fit(train_batch_generator, validation_data=val_batch_generator, 
                steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, 
                verbose=1, callbacks=[checkpoint, history, reduce_lr])

Epoch 1/25
21/21 [==============================] - 49s 2s/step - loss: 1.6234 - acc: 0.1994 - val_loss: 1.6398 - val_acc: 0.1719 - lr: 0.0100
Epoch 2/25
21/21 [==============================] - 45s 2s/step - loss: 1.5818 - acc: 0.1667 - val_loss: 1.3091 - val_acc: 0.1562 - lr: 0.0100
Epoch 3/25
21/21 [==============================] - 46s 2s/step - loss: 1.6253 - acc: 0.1756 - val_loss: 1.3005 - val_acc: 0.1875 - lr: 0.0100
Epoch 4/25
21/21 [==============================] - 47s 2s/step - loss: 1.5753 - acc: 0.1488 - val_loss: 1.6102 - val_acc: 0.2188 - lr: 0.0100
Epoch 5/25
21/21 [==============================] - 46s 2s/step - loss: 1.6116 - acc: 0.2054 - val_loss: 1.3245 - val_acc: 0.2969 - lr: 0.0100
Epoch 6/25
21/21 [==============================] - 46s 2s/step - loss: 1.5865 - acc: 0.1488 - val_loss: 1.2981 - val_acc: 0.2656 - lr: 0.0100
Epoch 7/25
21/21 [==============================] - 46s 2s/step - loss: 1.6120 - acc: 0.1935 - val_loss: 1.6127 - val_acc: 0.1719 - lr: 0.0100

In [ ]:
# we have 30 images, but we chose 16 below and we pick them randomly and sort to ensure temporal dependence
num_samples = 24
num_of_channels = 3
num_of_classes = 5
batch_size =16

img_width, img_height = 120, 160

# Let us try video classification using conv2D model + RNN - another method

input_size_tuple = (num_samples, img_width, img_height, num_of_channels)

from keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D

def build_convnet(shape=(img_width, img_height, num_of_channels)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape,
        padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())
    return model


In [ ]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout

def action_model(num_of_classes, shape=(num_samples, img_width, img_height, num_of_channels)):
    # Create our convnet with (img_width, img_height, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (number_of_samples, img_widht, img_height, num_of_channels) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_of_classes, activation='softmax'))
    return model

In [ ]:

model = action_model(num_of_classes, shape=(num_samples, img_width, img_height, num_of_channels))

'''
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)
'''

"\noptimizer = keras.optimizers.Adam(0.001)\nmodel.compile(\n    optimizer,\n    'categorical_crossentropy',\n    metrics=['acc']\n)\n"

In [ ]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_4 (TimeDis  (None, 24, 512)          4685376   
 tributed)                                                       
                                                                 
 gru_4 (GRU)                 (None, 64)                110976    
                                                                 
 dense_20 (Dense)            (None, 1024)              66560     
                                                                 
 dropout_12 (Dropout)        (None, 1024)              0         
                                                                 
 dense_21 (Dense)            (None, 512)               524800    
                                                                 
 dropout_13 (Dropout)        (None, 512)               0         
                                                      

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
history = History()

epochs = 20

# let us create checkpoint so that weights are saved after every epoch
checkpoint_filepath = ROOT_PATH + "/" + "ccn_checkpoint_8.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, save_weights_only=False)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.01)
opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['acc'])


In [ ]:
train_batch_generator =  batch_generator(new_train_map_df, batch_size, 'train', train_image_dict)
val_batch_generator =  batch_generator(new_val_map_df, batch_size, 'val', val_image_dict)

In [ ]:
model.fit(train_batch_generator, validation_data=val_batch_generator, 
          steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, verbose=1, callbacks=[checkpoint, history, reduce_lr])

Epoch 1/20
21/21 [==============================] - 58s 2s/step - loss: 1.6808 - acc: 0.2113 - val_loss: 1.6467 - val_acc: 0.1875 - lr: 0.0100
Epoch 2/20
21/21 [==============================] - 42s 2s/step - loss: 1.5875 - acc: 0.1607 - val_loss: 1.3149 - val_acc: 0.1719 - lr: 0.0100
Epoch 3/20
21/21 [==============================] - 42s 2s/step - loss: 1.6198 - acc: 0.1756 - val_loss: 1.2997 - val_acc: 0.3750 - lr: 0.0100
Epoch 4/20
21/21 [==============================] - 43s 2s/step - loss: 1.5761 - acc: 0.2054 - val_loss: 1.6119 - val_acc: 0.2188 - lr: 0.0100
Epoch 5/20
21/21 [==============================] - 42s 2s/step - loss: 1.6158 - acc: 0.2232 - val_loss: 1.3088 - val_acc: 0.2969 - lr: 0.0100
Epoch 6/20
21/21 [==============================] - 42s 2s/step - loss: 1.5719 - acc: 0.1488 - val_loss: 1.3064 - val_acc: 0.3281 - lr: 0.0100
Epoch 7/20
21/21 [==============================] - 42s 2s/step - loss: 1.6120 - acc: 0.1577 - val_loss: 1.6101 - val_acc: 0.2031 - lr: 0.0100

In [ ]:
loaded_model = keras.models.load_model(checkpoint_filepath)
loaded_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_4 (TimeDis  (None, 24, 512)          4685376   
 tributed)                                                       
                                                                 
 gru_4 (GRU)                 (None, 64)                110976    
                                                                 
 dense_20 (Dense)            (None, 1024)              66560     
                                                                 
 dropout_12 (Dropout)        (None, 1024)              0         
                                                                 
 dense_21 (Dense)            (None, 512)               524800    
                                                                 
 dropout_13 (Dropout)        (None, 512)               0         
                                                      

In [ ]:
epochs = 20
loaded_model.fit(train_batch_generator, validation_data=val_batch_generator, 
          steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, verbose=1, callbacks=[checkpoint, history, reduce_lr])

Epoch 1/20
21/21 [==============================] - 43s 2s/step - loss: 1.6131 - acc: 0.1726 - val_loss: 1.6084 - val_acc: 0.2031 - lr: 0.0100
Epoch 2/20
21/21 [==============================] - 43s 2s/step - loss: 1.5831 - acc: 0.1667 - val_loss: 1.6110 - val_acc: 0.1719 - lr: 0.0100
Epoch 3/20
21/21 [==============================] - 42s 2s/step - loss: 1.6174 - acc: 0.1994 - val_loss: 1.2952 - val_acc: 0.2500 - lr: 0.0100
Epoch 4/20
21/21 [==============================] - 42s 2s/step - loss: 1.5701 - acc: 0.2054 - val_loss: 1.3056 - val_acc: 0.1562 - lr: 0.0100
Epoch 5/20
21/21 [==============================] - 43s 2s/step - loss: 1.6124 - acc: 0.1935 - val_loss: 1.6084 - val_acc: 0.2188 - lr: 0.0100
Epoch 6/20
21/21 [==============================] - 42s 2s/step - loss: 1.5678 - acc: 0.1964 - val_loss: 1.3062 - val_acc: 0.1875 - lr: 0.0100
Epoch 7/20
21/21 [==============================] - 42s 2s/step - loss: 1.6105 - acc: 0.1756 - val_loss: 1.3016 - val_acc: 0.2656 - lr: 0.0100

In [ ]:
# we have 30 images, but we chose 12 below and we pick them randomly and sort to ensure temporal dependence
num_samples = 12
num_of_channels = 3
num_of_classes = 5
batch_size =16

img_width, img_height = 120, 160

# Let us try video classification using conv2D model + RNN - another method

input_size_tuple = (num_samples, img_width, img_height, num_of_channels)

from keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D

def build_convnet(shape=(img_width, img_height, num_of_channels)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape,
        padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())
    return model

In [ ]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout

def action_model(num_of_classes, shape=(num_samples, img_width, img_height, num_of_channels)):
    # Create our convnet with (img_width, img_height, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (number_of_samples, img_widht, img_height, num_of_channels) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_of_classes, activation='softmax'))
    return model

In [ ]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_4 (TimeDis  (None, 24, 512)          4685376   
 tributed)                                                       
                                                                 
 gru_4 (GRU)                 (None, 64)                110976    
                                                                 
 dense_20 (Dense)            (None, 1024)              66560     
                                                                 
 dropout_12 (Dropout)        (None, 1024)              0         
                                                                 
 dense_21 (Dense)            (None, 512)               524800    
                                                                 
 dropout_13 (Dropout)        (None, 512)               0         
                                                      

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
history = History()

epochs = 20

# let us create checkpoint so that weights are saved after every epoch
checkpoint_filepath = ROOT_PATH + "/" + "ccn_checkpoint_9.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, save_weights_only=False)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.01)
opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['acc'])

In [ ]:
model.fit(train_batch_generator, validation_data=val_batch_generator, 
          steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, verbose=1, callbacks=[checkpoint, history, reduce_lr])

Epoch 1/20
21/21 [==============================] - 37s 1s/step - loss: 1.6195 - acc: 0.2054 - val_loss: 1.6131 - val_acc: 0.1719 - lr: 0.0100
Epoch 2/20
21/21 [==============================] - 28s 1s/step - loss: 1.5707 - acc: 0.1964 - val_loss: 1.3034 - val_acc: 0.3125 - lr: 0.0100
Epoch 3/20
21/21 [==============================] - 28s 1s/step - loss: 1.6107 - acc: 0.1607 - val_loss: 1.2999 - val_acc: 0.3750 - lr: 0.0100
Epoch 4/20
21/21 [==============================] - 29s 1s/step - loss: 1.5732 - acc: 0.1964 - val_loss: 1.6083 - val_acc: 0.1875 - lr: 0.0100
Epoch 5/20
21/21 [==============================] - 28s 1s/step - loss: 1.6261 - acc: 0.1548 - val_loss: 1.3055 - val_acc: 0.2031 - lr: 0.0100
Epoch 6/20
21/21 [==============================] - 29s 1s/step - loss: 1.5742 - acc: 0.1577 - val_loss: 1.3018 - val_acc: 0.2656 - lr: 0.0100
Epoch 7/20
21/21 [==============================] - 29s 1s/step - loss: 1.6131 - acc: 0.1815 - val_loss: 1.6105 - val_acc: 0.1719 - lr: 0.0100

In [22]:
# we have 30 images, but we chose 28 images below and we pick them randomly and sort to ensure temporal dependence
num_samples = 28
num_of_channels = 3
num_of_classes = 5
batch_size =16

img_width, img_height = 120, 160

# Let us try video classification using conv2D model + RNN - another method

input_size_tuple = (num_samples, img_width, img_height, num_of_channels)

from keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D

def build_convnet(shape=(img_width, img_height, num_of_channels)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv2D(16, (3,3), input_shape=shape,
        padding='same', activation='relu'))
    model.add(Conv2D(16, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())
    return model

In [23]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout

def action_model(num_of_classes, shape=(num_samples, img_width, img_height, num_of_channels)):
    # Create our convnet with (img_width, img_height, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (number_of_samples, img_widht, img_height, num_of_channels) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(num_of_classes, activation='softmax'))
    return model

In [24]:
model = action_model(num_of_classes, shape=(num_samples, img_width, img_height, num_of_channels))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_1 (TimeDis  (None, 28, 128)          293520    
 tributed)                                                       
                                                                 
 gru_1 (GRU)                 (None, 64)                37248     
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                      

In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
history = History()

epochs = 20

# let us create checkpoint so that weights are saved after every epoch
checkpoint_filepath = ROOT_PATH + "/" + "ccn_checkpoint_10.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, save_weights_only=False)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.01)
opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['acc'])

In [ ]:
model.fit(train_batch_generator, validation_data=val_batch_generator, 
          steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, verbose=1, callbacks=[checkpoint, history, reduce_lr])

Epoch 1/20
21/21 [==============================] - 105s 4s/step - loss: 1.6507 - acc: 0.1905 - val_loss: 1.2553 - val_acc: 0.1641 - lr: 0.0100
Epoch 2/20
21/21 [==============================] - 87s 4s/step - loss: 1.5986 - acc: 0.1741 - val_loss: 1.2547 - val_acc: 0.1641 - lr: 0.0100
Epoch 3/20
21/21 [==============================] - 88s 4s/step - loss: 1.5885 - acc: 0.2188 - val_loss: 1.2547 - val_acc: 0.1641 - lr: 0.0100
Epoch 4/20
21/21 [==============================] - 86s 4s/step - loss: 1.5904 - acc: 0.2202 - val_loss: 1.2564 - val_acc: 0.1719 - lr: 0.0100
Epoch 5/20
16/21 [=====================>........] - ETA: 18s - loss: 1.6149 - acc: 0.2168

In [16]:
# we have 30 images, but we chose 28 images below and we pick them randomly and sort to ensure temporal dependence
num_samples = 28
num_of_channels = 3
num_of_classes = 5
batch_size = 32

img_width, img_height = 120, 160

# Let us try video classification using conv2D model + RNN - another method

input_size_tuple = (num_samples, img_width, img_height, num_of_channels)

from keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D

def build_convnet(shape=(img_width, img_height, num_of_channels)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv2D(32, (3,3), input_shape=shape,
        padding='same', activation='relu'))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))

    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())

    '''
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
  
    # flatten...
    model.add(GlobalMaxPool2D())
    '''
    return model

In [17]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout

def action_model(num_of_classes, shape=(num_samples, img_width, img_height, num_of_channels)):
    # Create our convnet with (img_width, img_height, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (number_of_samples, img_widht, img_height, num_of_channels) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(.2))
    #model.add(Dense(32, activation='relu'))
    #model.add(Dropout(.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(num_of_classes, activation='softmax'))
    return model

In [18]:
model = action_model(num_of_classes, shape=(num_samples, img_width, img_height, num_of_channels))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 28, 128)          287904    
 ibuted)                                                         
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                      

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.callbacks import History 
history = History()

epochs = 20

# let us create checkpoint so that weights are saved after every epoch
checkpoint_filepath = ROOT_PATH + "/" + "ccn_checkpoint_11.h5"
checkpoint = ModelCheckpoint(checkpoint_filepath, save_weights_only=False)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.1)
opt = keras.optimizers.Adam(learning_rate=0.1)

model.compile(loss=categorical_crossentropy, optimizer=opt, metrics=['acc'])

In [20]:
train_batch_generator =  batch_generator(new_train_map_df, batch_size, 'train', train_image_dict)
val_batch_generator =  batch_generator(new_val_map_df, batch_size, 'val', val_image_dict)

In [21]:
model.fit(train_batch_generator, validation_data=val_batch_generator, 
          steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, epochs=epochs, verbose=1, callbacks=[checkpoint, history, reduce_lr])

Epoch 1/20


ResourceExhaustedError: ignored